In [1]:
!pip install minsearch

In [2]:
import minsearch
import json 

In [3]:
with open('documents.json', 'rt') as  f_in:
    docs_raw = json.load(f_in)

In [4]:
documents = []

for course_dict in docs_raw:
    for doc in course_dict['documents']:
        doc['course'] = course_dict['course']
        documents.append(doc)

documents[0]

{'text': "The purpose of this document is to capture frequently asked technical questions\nThe exact day and hour of the course will be 15th Jan 2024 at 17h00. The course will start with the first  “Office Hours'' live.1\nSubscribe to course public Google Calendar (it works from Desktop only).\nRegister before the course starts using this link.\nJoin the course Telegram channel with announcements.\nDon’t forget to register in DataTalks.Club's Slack and join the channel.",
 'section': 'General course-related questions',
 'question': 'Course - When will the course start?',
 'course': 'data-engineering-zoomcamp'}

In [5]:
index = minsearch.Index(
    text_fields=["questions", 'text', 'section'],
    keyword_fields=['course']
)

In [6]:
# SELECT * WHERE course = 'data-engineering-zoomcamp'

In [7]:
index.fit(documents)

In [8]:
documents[0]

{'text': "The purpose of this document is to capture frequently asked technical questions\nThe exact day and hour of the course will be 15th Jan 2024 at 17h00. The course will start with the first  “Office Hours'' live.1\nSubscribe to course public Google Calendar (it works from Desktop only).\nRegister before the course starts using this link.\nJoin the course Telegram channel with announcements.\nDon’t forget to register in DataTalks.Club's Slack and join the channel.",
 'section': 'General course-related questions',
 'question': 'Course - When will the course start?',
 'course': 'data-engineering-zoomcamp'}

In [9]:
import os
from openai import OpenAI
from dotenv import load_dotenv

base_url=os.environ.get("BASE_URL")
api_key=os.environ.get("API_KEY")

client = OpenAI(
    base_url=base_url,
    api_key=api_key,
)

In [10]:
def search(query):
    boost = {'question': 3.0, 'section': 0.5}


    results = index.search(
        query=query,
        filter_dict={'course': 'data-engineering-zoomcamp'},
        boost_dict=boost,
        num_results=5
    )

    return results

In [11]:
def build_prompt(query, search_results):
    prompt_template = """
    Your are a course teaching assistant. 
    Answer the QUESTION based on the CONTEXT from the FAQ database. 
    Use only the facts from the CONTEXT  when answering the QUESTION. 
    If the CONTEXT doesn't containt the answer output None.

    QUESTION:
    {question}

    CONTEXT:
    {context}
    """.strip()
    context = ""

    for doc in search_results:
        context = context + f"section: {doc['section']}\nquestion:{doc['question']}\nanswer: {doc['text']}\n\n"

    prompt = prompt_template.format(question=query, context=context).strip()
    
    return prompt 


In [12]:
def llm(prompt):
    responce = client.chat.completions.create(
        model="deepseek/deepseek-chat-v3-0324:free",
        messages=[{
            "role":"user",
            "content":prompt
        }]
    )
    return responce.choices[0].message.content

In [13]:
query = "how do I run kafka?"

def rag(query):
    search_results = search(query)
    prompt = build_prompt(query=query, search_results=search_results)
    return llm(prompt=prompt)

In [14]:
rag(query)

'The context does not provide specific instructions on how to run Kafka. However, it mentions troubleshooting steps for common issues related to running Kafka (e.g., dependency errors, broker availability, etc.). \n\nIf you are looking to run Kafka, you would typically need to:\n1. Set up a Kafka broker (e.g., using Docker as mentioned in the `NoBrokersAvailable` error solution).\n2. Ensure dependencies are installed (e.g., `kafka-python`, `confluent-kafka`, etc.).\n3. Use Python or Java scripts to interact with Kafka (e.g., `stream.py`, `producer.py`, or Java `.jar` files).\n\nFor a complete guide on running Kafka, refer to official Kafka documentation or course materials not included in this context. \n\nOutput: None.'

In [15]:
documents[0]

{'text': "The purpose of this document is to capture frequently asked technical questions\nThe exact day and hour of the course will be 15th Jan 2024 at 17h00. The course will start with the first  “Office Hours'' live.1\nSubscribe to course public Google Calendar (it works from Desktop only).\nRegister before the course starts using this link.\nJoin the course Telegram channel with announcements.\nDon’t forget to register in DataTalks.Club's Slack and join the channel.",
 'section': 'General course-related questions',
 'question': 'Course - When will the course start?',
 'course': 'data-engineering-zoomcamp'}

In [20]:
from elasticsearch import Elasticsearch
es_client = Elasticsearch('http://localhost:9200') 

In [22]:
!curl http://localhost:9200

{
  "name" : "7691011013e3",
  "cluster_name" : "docker-cluster",
  "cluster_uuid" : "Ji-20tU_T56EYCNSzpyHOQ",
  "version" : {
    "number" : "8.4.3",
    "build_flavor" : "default",
    "build_type" : "docker",
    "build_hash" : "42f05b9372a9a4a470db3b52817899b99a76ee73",
    "build_date" : "2022-10-04T07:17:24.662462378Z",
    "build_snapshot" : false,
    "lucene_version" : "9.3.0",
    "minimum_wire_compatibility_version" : "7.17.0",
    "minimum_index_compatibility_version" : "7.0.0"
  },
  "tagline" : "You Know, for Search"
}
